# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [ ]:
# Instalamos el modelo predictivo Prophet
!python -m pip install prophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [ ]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [ ]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = datetime.date.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-USD',start_date, today).reset_index()

btc_df.tail()

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2828,2023-09-29,"$27,024.84","$27,225.94","$26,721.76","$26,911.72","$26,911.72",10396435377
2829,2023-09-30,"$26,911.69","$27,091.79","$26,888.97","$26,967.92","$26,967.92",5331172801
2830,2023-10-01,"$26,967.40","$28,047.24","$26,965.09","$27,983.75","$27,983.75",9503917434
2831,2023-10-02,"$27,976.80","$28,494.46","$27,347.79","$27,530.79","$27,530.79",19793041322
2832,2023-10-03,"$27,508.25","$27,667.19","$27,216.00","$27,429.98","$27,429.98",11407814187


In [ ]:
# TAREA: Validamos que no hay datos vacíos
btc_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = btc_df[['Date', 'Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
new_names = {
    "Date": "ds",
    "Open": "y",
}

# Los renombramos con los nuevos nombres
df.rename(columns=new_names, inplace=True)
df['ds'] = df['ds'].dt.tz_localize(None)

In [ ]:
df.tail()

,ds,y
2828,2023-09-29,"$27,024.84"
2829,2023-09-30,"$26,911.69"
2830,2023-10-01,"$26,967.40"
2831,2023-10-02,"$27,976.80"
2832,2023-10-03,"$27,508.25"


In [ ]:
# Crear la gráfica del precio de apertura

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título
fig.update_layout(
    title_text="Serie temporal de Bitcoin Precio de Apertura",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



Documentacion oficial de Prophet [GitHub](https://facebook.github.io/prophet/docs/quick_start.html)

In [ ]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode='multiplicative')

In [ ]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp21vhz3rn/2alwx9sd.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp21vhz3rn/5w3pll_4.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=82892', 'data', 'file=/tmp/tmp21vhz3rn/2alwx9sd.json', 'init=/tmp/tmp21vhz3rn/5w3pll_4.json', 'output', 'file=/tmp/tmp21vhz3rn/prophet_modelvyjzwosg/prophet_model-20231004104734.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:47:34 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:47:36 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [ ]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(365)
future.tail()

,ds
3193,2024-09-28
3194,2024-09-29
3195,2024-09-30
3196,2024-10-01
3197,2024-10-02


In [ ]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-01
1,2016-01-02
2,2016-01-03
3,2016-01-04
4,2016-01-05
...,...
3193,2024-09-28
3194,2024-09-29
3195,2024-09-30
3196,2024-10-01


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [ ]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2016-01-01,$22.62,"$-5,193.69","$4,768.31",$22.62,$22.62,$-0.01,$-0.01,$-0.01,$-0.00,$-0.00,$-0.00,$-0.01,$-0.01,$-0.01,$0.00,$0.00,$0.00,$22.33
1,2016-01-02,$25.19,"$-5,039.58","$5,137.31",$25.19,$25.19,$-0.01,$-0.01,$-0.01,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$0.00,$0.00,$0.00,$25.05
2,2016-01-03,$27.76,"$-5,077.03","$4,746.12",$27.76,$27.76,$0.00,$0.00,$0.00,$-0.00,$-0.00,$-0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$27.79
3,2016-01-04,$30.33,"$-5,047.89","$5,192.51",$30.33,$30.33,$0.01,$0.01,$0.01,$0.00,$0.00,$0.00,$0.01,$0.01,$0.01,$0.00,$0.00,$0.00,$30.55
4,2016-01-05,$32.90,"$-5,202.35","$5,312.76",$32.90,$32.90,$0.01,$0.01,$0.01,$0.00,$0.00,$0.00,$0.01,$0.01,$0.01,$0.00,$0.00,$0.00,$33.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3193,2024-09-28,"$18,089.74","$-1,518.20","$31,118.74","$-1,390.11","$36,958.09",$-0.18,$-0.18,$-0.18,$-0.00,$-0.00,$-0.00,$-0.18,$-0.18,$-0.18,$0.00,$0.00,$0.00,"$14,886.37"
3194,2024-09-29,"$18,077.33","$-1,218.27","$31,930.63","$-1,492.48","$37,064.70",$-0.17,$-0.17,$-0.17,$-0.00,$-0.00,$-0.00,$-0.17,$-0.17,$-0.17,$0.00,$0.00,$0.00,"$14,954.64"
3195,2024-09-30,"$18,064.92","$-1,753.83","$32,385.45","$-1,643.08","$37,171.30",$-0.17,$-0.17,$-0.17,$0.00,$0.00,$0.00,$-0.17,$-0.17,$-0.17,$0.00,$0.00,$0.00,"$15,046.30"
3196,2024-10-01,"$18,052.52",$-717.94,"$31,304.60","$-1,708.84","$37,277.91",$-0.16,$-0.16,$-0.16,$0.00,$0.00,$0.00,$-0.16,$-0.16,$-0.16,$0.00,$0.00,$0.00,"$15,119.82"


In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
3193,2024-09-28,"$14,886.37","$-1,518.20","$31,118.74"
3194,2024-09-29,"$14,954.64","$-1,218.27","$31,930.63"
3195,2024-09-30,"$15,046.30","$-1,753.83","$32,385.45"
3196,2024-10-01,"$15,119.82",$-717.94,"$31,304.60"
3197,2024-10-02,"$15,189.32","$-2,030.92","$32,311.24"


In [ ]:
next_day = (datetime.date.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

19399.82532979855

<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [ ]:
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)